In [18]:
import numpy as np
import math
import re
import itertools
import pandas as pd
import time
import os
import pickle
import copy
from Functions import *

In [19]:
N=6
dct_cmbn={}
dct_cmbn.update({1:np.arange(N)})
for j in range(2,5):
    dct_cmbn.update({j:np.array(list(itertools.combinations(np.arange(N),j)))})

In [20]:
def add_1(combinantions_dictionary, ND=5):
    N=combinantions_dictionary[1][-1]+1
    new_cd={1:np.append(combinantions_dictionary[1],N)}
    diff=1
    while diff<(ND-1):
        new_part=np.vstack([combinantions_dictionary[diff].T,np.array([N]*len(combinantions_dictionary[diff]))])
        new_in=np.hstack([combinantions_dictionary[diff+1].T,new_part]).T
        new_cd.update({(diff+1):new_in})

        diff+=1
    

    return(new_cd)


In [21]:
N=34
dct_cmbn={}
dct_cmbn.update({1:np.arange(N)})
for j in range(2,5):
    dct_cmbn.update({j:np.array(list(itertools.combinations(np.arange(N),j)))})

In [22]:
F_dict={6:dct_cmbn}

In [23]:
def add_N(full_dict,N_start, N_add):
    tmp_d=full_dict[N_start]
    i=0
    while i<N_add:
        tmp_d=add_1(tmp_d)
        full_dict.update({(N_start+i+1):tmp_d})
        i+=1

    return(full_dict)


In [24]:
def iterative_add_N(dict_start, N_add, save=True,save_dir='./combinations/'):
    tmp_d=copy.deepcopy(dict_start)
    N_start=dict_start[1][-1]
    i=0
    while i<N_add:
        print(i)
        tmp_d=add_1(tmp_d)
        if save:
            NM=save_dir+'N_'+str(N_start+i+1)+'.pk'
            with open(NM, 'wb') as handle:
                pickle.dump(tmp_d, handle, protocol=pickle.HIGHEST_PROTOCOL)
        i+=1

    return(tmp_d)

In [25]:
WA_mat=assign_wells_mat(n_compounds=35)


In [26]:
WA_mat.shape

(35, 12)

In [27]:
dct_cmbn[4].shape

(46376, 4)

In [28]:
f1=add_1(dct_cmbn)

In [29]:
f1[4].shape

(52360, 4)

In [30]:
WA_mat[f1[4]].shape

(52360, 4, 12)

In [31]:
sm=np.bool(np.sum(WA_mat[f1[4]], axis=1))

In [32]:
sm[0,:]

array([ True, False, False, False, False, False,  True,  True,  True,
        True, False, False])

In [ ]:
sm.shape

In [5]:

#NMO=save_dir+'N_'+str(N)+'.pk'
NMO='./combinations/N_294.pk'
with open(NMO, "rb") as input_file:
    scrambler = pickle.load(input_file)

In [ ]:
def is_consistent_precomp(well_assigner:np.array, differentiate:int, scrambler:dict) -> list:
    if differentiate==0:
        return(True,well_assigner, np.array([1]*well_assigner.shape[0]))
    N=well_assigner.shape[0]
    for i in range(differentiate):
        diff=i+1
        if diff ==1:
            full_well_assigner=well_assigner.copy()
        else:
            this_sc=scrambler[diff]
            full_well_assigner=np.concatenate((full_well_assigner,np.bool(np.sum(well_assigner[this_sc], axis=1))))
    _, counts=np.unique(full_well_assigner, axis=0, return_counts=True)
    if np.unique(full_well_assigner, axis=0).shape[0]<full_well_assigner.shape[0]:
        return(False, full_well_assigner, counts)
    elif np.unique(full_well_assigner, axis=0).shape[0]==full_well_assigner.shape[0]:
        return(True,full_well_assigner, counts)
    else:
        print("Something is fishy")
        return(-1)
    
#


In [ ]:
def decode_precomp(well_assigner:np.ndarray, readout:np.ndarray, differentiate:int, scrambler:dict) -> list:
    N=well_assigner.shape[0]
    for i in range(differentiate):
        resulti=[]
        diff=i+1
        if diff ==1:
            resulti.extend(list(range(well_assigner.shape[0])))
            full_well_assigner=well_assigner.copy()
        else:
            this_sc=scrambler[diff]
            full_well_assigner=np.concatenate((full_well_assigner,np.bool(np.sum(well_assigner[this_sc], axis=1))))
    idxs=[i for i in range(full_well_assigner.shape[0]) if np.array_equal(full_well_assigner[i,:],readout)]
    if len(idxs)==0:
        print('No match')
        return(-1)
    
    return [resulti[i] for i in idxs]

In [ ]:
def full_deterministic_comparison(**kwargs):
    methods=['matrix', 'random', 'STD', 'Chinese trick']
    # matrix assignment
    

    # multidimensional matrix
    if 'n_dims' in kwargs.keys():
        WA_mul=assign_wells_multidim(**kwargs)
        ndmin=kwargs['n_dims']
        multi=['multidim: '+str(ndmin)]
        WA_list=[WA_mul]
    elif 'all_dims' in kwargs.keys():
        if kwargs['all_dims']:
            WA_list=[]
            multi=[]
            for i in np.arange(2,int(np.ceil(np.log(kwargs['n_compounds'])/np.log(2)))):
                if i>kwargs['max_dims']:
                    continue
                WA_mul=assign_wells_multidim(n_dims=i, **kwargs)
                WA_list.append(WA_mul)
                multi.append('multidim: '+str(i))

        else:
            ndmin= find_dims(**kwargs)
            WA_mul=assign_wells_multidim(n_dims=ndmin, **kwargs)
            multi=['multidim: '+str(ndmin)]
            WA_list=[WA_mul]
            


    else:
        ndmin= find_dims(**kwargs)
        WA_mul=assign_wells_multidim(n_dims=ndmin, **kwargs)
        multi=['multidim: '+str(ndmin)]
        WA_list=[WA_mul]
    
    multi.extend(methods)
    methods=multi.copy()

    WA_mat=assign_wells_mat(**kwargs)

    # random assignment

    WA_ran=assign_wells_random(**kwargs)

    # STD asignment 
    WA_std=assign_wells_STD(**kwargs)

    

    # chinese trick assignment
    WA_chin=assign_wells_chinese(**kwargs)


    WA_list.extend([WA_mat, WA_ran,WA_std, WA_chin])

    if kwargs['differentiate']<2:
        WA_bin=assign_wells_L(**kwargs)
        methods.append('Binary')
        WA_list.append(WA_bin)

    #hierarchical
        
    Hier=calculate_metrics_hierarchical(**kwargs)
    #return([BM[0], BM[1],layers, MC, details ])

    ls_met=[]
    ls_names_met=['mean_experiments', 'max_compounds_per_well', 'n_wells', 'percentage_check', 'mean_extra_exp', 'mean_steps']
    for method, WA in zip(methods, WA_list):
        mean_exp, extra_exp,  _, perc_check= mean_metrics(WA, **kwargs)
        n_wells=WA.shape[1]
        M_exp=np.round(mean_exp, 2)
        max_comp=np.max(np.sum(WA, axis=0))
        ls_met.append([M_exp, max_comp, n_wells, int(perc_check),  extra_exp,1+perc_check/100])
    ls_met.append(Hier[:-1])
    full_methods=methods.copy()
    full_methods.append('Hierarchical')
    df_met=pd.DataFrame(ls_met)

    dict_wa={method: WA for method, WA in zip(methods, WA_list)}
    dict_wa.update({'Hierarchical':Hier[5]})

    idx_renamer={i:j for i,j in zip(df_met.index, full_methods)}
    col_renamer={i:j for i,j in zip(df_met.columns, ls_names_met)}
    df_met.rename(index=idx_renamer, columns=col_renamer, inplace=True)

    ret_wa= kwargs['return_wa'] 
    if ret_wa:
        return df_met, dict_wa
    return df_met

In [ ]:
with open('./combinations/N_294.pk', 'rb') as handle:
    f1=pickle.load(handle)

In [ ]:
F2=iterative_add_N(f1,400, save=False)

In [ ]:
F1=add_N(F_dict,6,200)

In [ ]:
with open('./cmb_dict.pk', 'wb') as handle:
    pickle.dump(F1, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
len(F1)

In [ ]:
F1[1000][4].shape

In [ ]:
len(list(itertools.combinations(np.arange(1000),4)))

In [ ]:
tri=add_1(dct_cmbn)

In [ ]:
tri

In [ ]:
np.array(list(itertools.combinations(np.arange(6),2))).shape

In [ ]:
pk_dir='./diff_1/20-47_step5.pk'

In [ ]:
pk_dir='./single_method/multidim_3/multidim_3__20-47_step5.pk'

In [ ]:
with open(pk_dir, 'rb') as handle:
    f1=pickle.load(handle)

In [ ]:
f1[45]['WA'].shape

In [ ]:
# To merge .pk files

ls_names_met=['mean_experiments', 'max_compounds_per_well', 'n_wells', 'percentage_check', 'mean_extra_exp']
full_dict={}
for j in range(1,5):
    base_dir='diff_'+str(j)
    diff_dict={}
    filenames = next(os.walk(base_dir), (None, None, []))[2]
    for file in filenames:
        full_dir=os.path.join(base_dir,file)
        with open(full_dir, 'rb') as handle:
            f1=pickle.load(handle)
        for i in list(f1):
            if i=='kwargs':
                del f1['kwargs']
                continue
            else:
                WA=assign_wells_mat(i)
                f1[i][1].update({'matrix':WA})
                mean_exp, extra_exp,  _, perc_check= mean_metrics(WA, differentiate=j)
                n_wells=WA.shape[1]
                M_exp=np.round(mean_exp, 2)
                max_comp=np.max(np.sum(WA, axis=0))
                ls_met=[M_exp, max_comp, n_wells, perc_check,  extra_exp]
                f2=f1[i][0].drop(labels='matrix')
                idxs=list(f2.index)
                idxs+=['matrix']
                r_idxs=idxs[-1:]+idxs[:-1]
                dict_met={nm:list(f2[nm])+[val] for nm,val in zip(ls_names_met, ls_met)}

                f1[i][0]=pd.DataFrame(data=dict_met, index=idxs).reindex(r_idxs)

                
                f1[i][0]=f1[i][0].round({'mean_extra_exp':2})
            
        diff_dict.update(f1)
    full_str='Differentiate '+str(j)
    full_dict.update({full_str:diff_dict})

    
    


In [ ]:
#output as pickle file
full_dir='Final_precomputed_file.pk'

#with open(full_dir, 'wb') as handle:
    #pickle.dump(full_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# To merge files generated with inline_print
all_methods = ['matrix', 'multidim', 'random', 'STD', 'CT', 'Binary']
results = {}

for method_dir in all_methods:
    filenames = next(os.walk(method_dir), (None, None, []))[2]
    metrics_list = []
    for file in filenames:
        if file.endswith(".txt"):
            try:
                metrics = extract_metrics(file)
                metrics["method"] = method_dir
                metrics_list.append(metrics)
            except ValueError as e:
                print(e)
    
    if metrics_list:
        df = pd.DataFrame(metrics_list)
        df = df.sort_values(by="NS")
        
        # Group by diff value
        for diff_value, group_df in df.groupby("diff"):
            group_df = group_df.reset_index(drop=True)
            
            if diff_value not in results:
                results['Differentiate ' + str(diff_value)] = {}
                
            for ns_value, ns_group_df in group_df.groupby("NS"):
                ns_group_df = ns_group_df.drop(columns=["diff", "NS"]) 
                ns_group_df = ns_group_df.rename(columns={"NW": "n_wells", "MS": "max_compounds_per_well"})
               
                ns_group_df = ns_group_df.set_index(["method"])
                
                if ns_value not in results['Differentiate ' + str(diff_value)]:
                    results['Differentiate ' + str(diff_value)][ns_value] = [ns_group_df, {}]
                else:
                    existing_df, empty_dict = results['Differentiate ' + str(diff_value)][ns_value]
                    
                    merged_df = pd.merge(existing_df.reset_index(), ns_group_df.reset_index(),
                                         on=["method", "n_wells", "max_compounds_per_well"], how="outer", indicator=True)
                    
                    # Filter out rows that are duplicates (the '_merge' column indicates if it's from both dataframes)
                    new_entries_df = merged_df[merged_df['_merge'] == 'right_only'].drop(columns=["_merge"]).set_index("method")
                    existing_entries_df = merged_df[merged_df['_merge'] == 'both'].drop(columns=["_merge"]).set_index("method")
                    
                    # Concatenate only new entries
                    combined_df = pd.concat([existing_entries_df, new_entries_df]).drop_duplicates().reset_index(drop=True)
                    
                    # Reassign the combined dataframe
                    results['Differentiate ' + str(diff_value)][ns_value] = [combined_df, {}]
            

In [ ]:

# Output as pickle file
full_dir = 'Final_inline_precomputed_file.pk'

# First check if file already exists
# Merge new and old if it does
if os.path.exists(full_dir):
    with open(full_dir, "rb") as f:
        existing_results = pickle.load(f)
        
    for diff_value, ns_values in results.items():
        if diff_value not in existing_results:
            existing_df, existing_empty_dict = existing_results[diff_value][ns_value]
            
            for ns_value, (new_df, empty_dict) in ns_values.items():
                if ns_value in existing_results[diff_value]:
                    existing_df, existing_empty_dict = existing_results[diff_value][ns_value]
                
            
                    merged_df = pd.merge(existing_df.reset_index(), new_df.reset_index(),
                                         on=["method", "n_wells", "max_compounds_per_well"], how="outer", indicator=True)
                    
                    new_entries_df = merged_df[merged_df['_merge'] == 'right_only'].drop(columns=["_merge"]).set_index("method")
                    existing_entries_df = merged_df[merged_df['_merge'] == 'both'].drop(columns=["_merge"]).set_index("method")
                    
                    combined_df = pd.concat([existing_entries_df, new_entries_df]).drop_duplicates().reset_index(drop=True)
                    existing_results[diff_value][ns_value] = [combined_df.set_index(["method"]), {}]
                else:
                    existing_results[diff_value][ns_value] = [new_df, empty_dict]
            
                
    merged_results = existing_results
    
else:
    merged_results = results

#with open(full_dir, 'wb') as handle:
    #pickle.dump(merged_results, handle, protocol=pickle.HIGHEST_PROTOCOL)